In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import requests
import numpy as np
from config import omdb_key
from config import tmdb_key

# Movies
## Create DataFrame and CSV of Movie Titles

In [2]:
file_path = 'Resources/Data_from_web/title.basics.tsv'
movie_titles_df = pd.read_csv(file_path, sep='\t')

print(movie_titles_df.columns)
movie_titles_df.head(2)

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


In [3]:
movie_titles_clean = movie_titles_df[['tconst', 'titleType', 'primaryTitle', 'isAdult', 
                                      'startYear', 'genres']]

# Get rid of '\N' values in startYear
movie_titles_clean = movie_titles_clean[movie_titles_clean['startYear'] != r'\N']

# need to convert type to int & string
movie_titles_clean = movie_titles_clean.astype({'startYear': int,
                                                'titleType': 'string'})

# filter language based on Englsih 2015-2019
movie_titles_clean = movie_titles_clean[(movie_titles_clean.startYear > 2014) & 
                                        (movie_titles_clean.startYear < 2020)]

# weed out adult  films & narrow down results to movies
movie_titles_clean = movie_titles_clean[movie_titles_clean['titleType'] == 'movie']
movie_titles_clean = movie_titles_clean[movie_titles_clean['isAdult'] == 0]

# remove any potential duplicates (tconst column)
movie_titles_clean = movie_titles_clean.drop_duplicates(subset=['tconst'])


In [4]:
print(f"THIS IS HOW MANY MOVIES: {movie_titles_clean.startYear.count()}")
movie_titles_clean.head(2)

THIS IS HOW MANY MOVIES: 84294


,tconst,titleType,primaryTitle,isAdult,startYear,genres
67687,tt0069049,movie,The Other Side of the Wind,0,2018,Drama
70840,tt0072306,movie,Toula ou Le génie des eaux,0,2017,Drama


# OMDB
### Pulling in data & preparign data for pull
* Due to the large number of movies we wished to query, we divided it down into pulling per year

In [11]:
OMDB_titles_df = movie_titles_clean.copy()

# Due to the large number of titles, we broke this pull into pulling per year
# CHANGE YEAR PER PULL
year = 2019
OMDB_titles_df = OMDB_titles_df[(OMDB_titles_df.startYear == year)]

,tconst,titleType,primaryTitle,isAdult,startYear,genres
114398,tt0116991,movie,Mariette in Ecstasy,0,2019,Drama
264758,tt0276132,movie,The Fetishist,0,2019,Animation


In [12]:
# Cleaning the data for OMDB pull

OMDB_titles_clean_df = OMDB_titles_df[['tconst', 'primaryTitle', 'startYear']]

# Make columns to import info into
OMDB_titles_clean_df['Metascore'] = " "
OMDB_titles_clean_df['imdbRating'] = " "
OMDB_titles_clean_df['imdbVotes'] = " "
OMDB_titles_clean_df['Title'] = " "


print(f"How many movies in {year} = {OMDB_titles_clean_df.primaryTitle.count()}")

OMDB_titles_clean_df.head(2)

How many movies in 2019 = 16352


,tconst,primaryTitle,startYear,Metascore,imdbRating,imdbVotes,Title
114398,tt0116991,Mariette in Ecstasy,2019,,,,
264758,tt0276132,The Fetishist,2019,,,,


### Pull OMDB

In [13]:
# # make URL
url_omdb = "http://www.omdbapi.com/?apikey="+ omdb_key + "&i="

error_count = 0

for index, row in OMDB_titles_clean_df.iterrows(): 
    try:
        movie_data = requests.get(url_omdb + str(movie_titles_clean.tconst[index])).json()
        try:
            OMDB_titles_clean_df.loc[index, 'Metascore'] = movie_data['Metascore']
            OMDB_titles_clean_df.loc[index, 'imdbRating'] = movie_data['imdbRating']
            OMDB_titles_clean_df.loc[index, 'imdbVotes'] = movie_data['imdbVotes']
            OMDB_titles_clean_df.loc[index, 'Title'] = movie_data['Title']        
        except (IndexError, KeyError, ValueError):
            error_count +=1
    # Added for OMDB errors when their system returns JSONDecodeError
    except(ValueError, TypeError):
        error_count +=1   

### Save data as CSV

In [16]:
file_outpath = f"Resources/OMDB_pull_{year}_error_count{error_count}.csv"

OMDB_titles_clean_df.to_csv(file_outpath)
OMDB_titles_clean_df.head(2)

,tconst,primaryTitle,startYear,Metascore,imdbRating,imdbVotes,Title
114398,tt0116991,Mariette in Ecstasy,2019,N/A,7.8,26,Mariette in Ecstasy
264758,tt0276132,The Fetishist,2019,N/A,N/A,N/A,The Fetishist


# TMDB
### Pulling in data & preparing data for pull

In [15]:
# CHANGE YEAR FOR PULL 
year = 2019

TMDB_movie_titles_df = movie_titles_clean.copy()

TMDB_movies_df = TMDB_movie_titles_df[['tconst', 'primaryTitle', 'startYear']]
TMDB_movies_df = TMDB_movies_df[(TMDB_movies_df.startYear == year)]

beginning_number = TMDB_movies_df.tconst.count()
TMDB_movies_df.head(2)


,tconst,primaryTitle,startYear
114398,tt0116991,Mariette in Ecstasy,2019
264758,tt0276132,The Fetishist,2019


In [17]:
# change title name to have + instead of ' '
TMDB_movies_df['primaryTitle'] = TMDB_movies_df['primaryTitle'].str.replace(" ", "+")


# ******Error 1: need to remove # from the beginning of titles for TMDB to work

# variable cause starswith() wasn't happy with '#'
pound_sign = '#'

# make dataframe for pound sign = True (startswith() returns True/False)
replace_pound_df = TMDB_movies_df.iloc[:, 0:3]
replace_pound_df.primaryTitle = replace_pound_df.primaryTitle.str.startswith(pound_sign)

# make df for ONLY the True values + primaryTitle from TMDB_movies_df
pound_true_df = replace_pound_df.loc[replace_pound_df.primaryTitle == True]
pound_true_df['TITLE'] = TMDB_movies_df['primaryTitle']

# Fix titles to not have # in the front & clean up columns
pound_true_df['TITLE'] = pound_true_df['TITLE'].str.replace(pound_sign, "")
pound_true_clean_df = pound_true_df.drop(columns=['primaryTitle', 'startYear'])
pound_true_clean_df = pound_true_clean_df.rename(columns={'TITLE': 'primaryTitle'})

# Merge 2 dfs, replace blank primaryTitle_y values with na so you can do fillna into a 
# nice new clean has correct info column & delete primaryTitle_y/x
titles_combined_df = pd.merge(TMDB_movies_df, pound_true_clean_df, how='outer', on='tconst')
titles_combined_df['primaryTitle_y'] = titles_combined_df['primaryTitle_y'].str.replace(" ", "nan")
titles_combined_df["primaryTitle"] = titles_combined_df["primaryTitle_y"].fillna(titles_combined_df["primaryTitle_x"])
titles_fixed_df = titles_combined_df.drop(columns=['primaryTitle_y', 'primaryTitle_x'])


# FINALLY make movie titles into a list so you can run it
movies = titles_fixed_df['primaryTitle'].tolist()


### TMDB pull 1 for TMDB IDs

In [ ]:
url_tmdb_id = "https://api.themoviedb.org/3/search/movie?api_key=" + tmdb_key + "&query="

response_tmdb_id = []
str_year = "&y=" + str(year)

error_count = 0

for movie in movies: 
    try: 
        movie_data = requests.get(url_tmdb_id + movie + str_year).json()
        if (movie_data['total_results'] == 1):
            response_tmdb_id.append(movie_data['results'][0]['id'])       
        else:
            error_count += 1
    except (IndexError, KeyError, ValueError):
        error_count += 1

### Save data as CSV

In [ ]:
file_outpath = f"Resources/TMDB_pull_1_{year}_error_count{error_count}.csv"

TMDB_df = pd.DataFrame(response_tmdb_id,columns=['ID'],dtype=object)
TMDB_df.to_csv(file_outpath)
TMDB_df.head(2)

### TMDB pull 2 for movie information

In [ ]:
url_tmdb_movie = "https://api.themoviedb.org/3/movie/"

# Make columns to import info into
TMDB_df['imdb_id'] = " "
TMDB_df['release_date'] = " "
TMDB_df['budget'] = " "
TMDB_df['revenue'] = " "
TMDB_df['genres'] = " "
TMDB_df['original_language'] = " "
TMDB_df['original_title'] = " "
TMDB_df['origin_country'] = " "
TMDB_df['production_countries name'] = " "
TMDB_df['spoken_languages name'] = " "
TMDB_df.head(2)

In [ ]:
error_count_info = 0

for index, row in TMDB_df.iterrows(): 
    movie_data = requests.get(url_tmdb_movie + str(TMDB_df.ID[index]) + "?api_key=" + tmdb_key).json()
    try:
        TMDB_df.loc[index, 'imdb_id'] = movie_data['imdb_id']
        TMDB_df.loc[index, 'release_date'] = movie_data['release_date']
        TMDB_df.loc[index, 'budget'] = movie_data['budget']
        TMDB_df.loc[index, 'revenue'] = movie_data['revenue']
        TMDB_df.loc[index, 'original_language'] = movie_data['spoken_languages'][0]['name']
        TMDB_df.loc[index, 'original_title'] = movie_data['original_title']
        TMDB_df.loc[index, 'origin_country'] = movie_data['production_countries'][0]['iso_3166_1']
        TMDB_df.loc[index, 'production_countries name'] = movie_data['production_countries'][0]['name']
        TMDB_df.loc[index, 'spoken_languages name'] = movie_data['spoken_languages'][0]['name']
        TMDB_df.loc[index, 'genres'] = movie_data['genres'][0]['name']    
    except (IndexError, KeyError):
        error_count_info +=1

### Save data as CSV

In [ ]:
file_outpath_2 = f"Resources/TMDB_pull_2_{year}_error_count{error_count_info}.csv"

TMDB_df.to_csv(file_outpath_2)
TMDB_df.head(2)

### CLEANING if budget = 0, revenue = 0, IMDB_id not found
* This is to help keep the file size down by dropping rows we cannot use or cannot match up

In [ ]:
movie_info_pulled_df = TMDB_df.copy()
movie_info_pulled_df.head()

movie_info_pulled_df = movie_info_pulled_df[movie_info_pulled_df.budget != 0]
movie_info_pulled_df = movie_info_pulled_df[movie_info_pulled_df.revenue != 0]
movie_info_pulled_df = movie_info_pulled_df.dropna(subset=['imdb_id'])

final_number = movie_info_pulled_df.imdb_id.count()

#### Save results as a CSV

In [ ]:
total_errors = beginning_number - final_number

file_outpath_FINAL = f"Resources/TMDB_pull_FINAL_{year}_dropped_movies_{total_errors}.csv"
movie_info_pulled_df.to_csv(file_outpath_FINAL)
movie_info_pulled_df.head(2)



# Netflix
## Create DataFrame and CSV of Netflix Titles

In [ ]:
# https://www.kaggle.com/abhimanyudasarwar/netflix-originals

file_path = 'Resources/netflix_originals.csv'

netflix_titles_df = pd.read_csv(file_path)

print(netflix_titles_df.columns)
netflix_titles_df.head(2)

In [ ]:
netflix_titles_clean = netflix_titles_df[['Title', 'Genre', 'Premiere', 'Seasons']]

list_genre =('Making-of', 'Aftershow / Interview','Musical / Short')

# remove 1 rouge string value in year
netflix_titles_clean = netflix_titles_clean[netflix_titles_clean['Seasons'].isnull()]
netflix_titles_clean = netflix_titles_clean[netflix_titles_clean['Genre'] != list_genre]
netflix_titles_clean = netflix_titles_clean[~netflix_titles_clean['Premiere'].str.contains('2020', na=False)]
netflix_fix_titles = netflix_titles_clean.reset_index()

print(netflix_fix_titles.Title.count())
netflix_fix_titles.head(2)

In [ ]:
# variable cause starswith() wasn't happy with '#'
pound_sign = '#'

# make dataframe for pound sign = True (startswith() returns True/False)
replace_pound_df = netflix_fix_titles.iloc[:, 0:2]
replace_pound_df.Title = replace_pound_df.Title.str.startswith(pound_sign)

# make df for ONLY the True values + primaryTitle from netflix_fix_titles
pound_true_df = replace_pound_df.loc[replace_pound_df.Title == True]
pound_true_df['TITLE'] = netflix_fix_titles['Title']

# Fix titles to not have # in the front & clean up columns
pound_true_df['TITLE'] = pound_true_df['TITLE'].str.replace(pound_sign, "")
pound_true_clean_df = pound_true_df.drop(columns=['Title'])
pound_true_clean_df = pound_true_clean_df.rename(columns={'TITLE': 'Title'})

# Merge 2 dfs, replace blank primaryTitle_y values with na so you can do fillna into a 
# nice new clean has correct info column & delete primaryTitle_y/x
titles_combined_df = pd.merge(netflix_fix_titles, pound_true_clean_df, how='outer', on='index')
titles_combined_df['Title_y'] = titles_combined_df['Title_y'].str.replace(" ", "nan")
titles_combined_df["Title"] = titles_combined_df["Title_y"].fillna(titles_combined_df["Title_x"])
titles_fixed_df = titles_combined_df.drop(columns=['Title_y', 'Title_x'])

print(titles_fixed_df.Title.count())
titles_fixed_df.head(2)

In [ ]:
beginning_number = titles_fixed_df['index'].count()

titles_fixed_df['Title'] = titles_fixed_df['Title'].str.replace(" ", "+")
            
print(f"\nTHIS IS HOW MANY MOVIES: {titles_fixed_df.Title.count()}")
titles_fixed_df.head(2)

# OMDB

In [ ]:
# make URL
url_omdb = "http://www.omdbapi.com/?apikey="+ omdb_key + "&t="

#make OMDB dataframe
OMDB_netflix_df = netflix_titles_clean.copy()
OMDB_netflix_df['Metascore'] = ''
OMDB_netflix_df['imdbRating'] = ''
OMDB_netflix_df['imdbVotes'] = ''
OMDB_netflix_df['Title_from_OMDB'] = ''
OMDB_netflix_df['imdbID'] = ''


error_count = 0

for index, row in OMDB_netflix_df.iterrows():           
    try:
        movie_data = requests.get(url_omdb + str(OMDB_netflix_df.Title[index])).json()
        try:
            OMDB_netflix_df.loc[index, 'Metascore'] = movie_data['Metascore']
            OMDB_netflix_df.loc[index, 'imdbRating'] = movie_data['imdbRating']
            OMDB_netflix_df.loc[index, 'imdbVotes'] = movie_data['imdbVotes']
            OMDB_netflix_df.loc[index, 'Title_from_OMDB'] = movie_data['Title']
            OMDB_netflix_df.loc[index, 'imdbID'] = movie_data['imdbID']
        except (IndexError, KeyError, ValueError):
            error_count +=1
    # Added for OMDB errors when their system returns JSONDecodeError
    except(ValueError, TypeError):
        error_count +=1    

### Save data as CSV

In [ ]:
file_outpath = f"Resources/OMDB_pull_Netflix_error_count{error_count}.csv"

OMDB_netflix_df.to_csv(file_outpath)
OMDB_netflix_df.head(2)

# TMDB
### Pulling in data & preparing data for pull

In [ ]:
# Make movie titles into a list so you can run it in TMDB pull 1
movies = titles_fixed_df['Title'].tolist()

### TMDB pull 1 for TMDB IDs

In [ ]:
imdb_id_url =  "https://api.themoviedb.org/3/movie/"

response_tmdb_id = []

error_count_TMDB = 0

for index, row in OMDB_netflix_df.iterrows():  
    movie_data = requests.get(imdb_id_url + OMDB_netflix_df.imdbID[index] + "/external_ids?api_key=" + tmdb_key).json()
    try:
        response_tmdb_id.append(movie_data['id']) 
    except:
        error_count_TMDB += 1    

#### Save results as a CSV

In [ ]:
file_outpath = f"Resources/TMDB_pull_1_Netflix_error_count{error_count_TMDB}.csv"

TMDB_neflix_1_df = pd.DataFrame(response_tmdb_id,columns=['ID'],dtype=object)
TMDB_neflix_1_df.to_csv(file_outpath)
TMDB_neflix_1_df.head(2)

### TMDB pull 2 for movie information

In [ ]:
#Make TMDB dataframe
TMDB_netflix_2_df = TMDB_neflix_1_df.copy()

url_tmdb_movie = "https://api.themoviedb.org/3/movie/"

# Make columns to import info into
TMDB_netflix_2_df['imdb_id'] = " "
TMDB_netflix_2_df['release_date'] = " "
TMDB_netflix_2_df['budget'] = " "
TMDB_netflix_2_df['revenue'] = " "
TMDB_netflix_2_df['genres'] = " "
TMDB_netflix_2_df['original_language'] = " "
TMDB_netflix_2_df['original_title'] = " "
TMDB_netflix_2_df['origin_country'] = " "
TMDB_netflix_2_df['production_countries name'] = " "
TMDB_netflix_2_df['spoken_languages name'] = " "

TMDB_netflix_2_df.head(2)

In [ ]:
error_count_info = 0

for index, row in TMDB_neflix_1_df.iterrows(): 
    movie_data = requests.get(url_tmdb_movie + str(TMDB_neflix_1_df.ID[index]) + "?api_key=" + tmdb_key).json()
    try:
        TMDB_netflix_2_df.loc[index, 'imdb_id'] = movie_data['imdb_id']
        TMDB_netflix_2_df.loc[index, 'release_date'] = movie_data['release_date']
        TMDB_netflix_2_df.loc[index, 'budget'] = movie_data['budget']
        TMDB_netflix_2_df.loc[index, 'revenue'] = movie_data['revenue']
        TMDB_netflix_2_df.loc[index, 'original_language'] = movie_data['spoken_languages'][0]['name']
        TMDB_netflix_2_df.loc[index, 'original_title'] = movie_data['original_title']
        TMDB_netflix_2_df.loc[index, 'origin_country'] = movie_data['production_countries'][0]['iso_3166_1']
        TMDB_netflix_2_df.loc[index, 'production_countries name'] = movie_data['production_countries'][0]['name']
        TMDB_netflix_2_df.loc[index, 'spoken_languages name'] = movie_data['spoken_languages'][0]['name']
        TMDB_netflix_2_df.loc[index, 'genres'] = movie_data['genres'][0]['name']    
    except (ValueError):
        error_count_info +=1

#### Save results as a CSV

In [ ]:
file_outpath_2 = f"Resources/TMDB_pull_2_Netflix_error_count{error_count_info}.csv"

TMDB_netflix_2_df.to_csv(file_outpath_2)
TMDB_netflix_2_df.head(2)

### CLEANING if budget = 0, revenue = 0, IMDB_id not found
* This is to help keep the file size down by dropping rows we cannot use or cannot match up

In [ ]:
neflix_info_pulled_df = TMDB_netflix_2_df.copy()
neflix_info_pulled_df.head(2)

neflix_info_pulled_df = neflix_info_pulled_df[neflix_info_pulled_df.budget != 0]
neflix_info_pulled_df = neflix_info_pulled_df[neflix_info_pulled_df.revenue != 0]
neflix_info_pulled_df = neflix_info_pulled_df.dropna(subset=['imdb_id'])

final_number = movie_info_pulled_df.imdb_id.count()

#### Save results as a CSV

In [ ]:
total_errors = beginning_number - final_number

file_outpath_FINAL = f"Resources/TMDB_pull_FINAL_Netflix_dropped_movies_{total_errors}.csv"
neflix_info_pulled_df.to_csv(file_outpath_FINAL)
neflix_info_pulled_df.head(2)

# Cleaning Pulled Data

In [ ]:
# import files
movies_2015 = pd.read_csv("Resources/2015_pull/TMDB_pull_FINAL_2015_dropped_movies_15964.csv")
movies_2016 = pd.read_csv("Resources/2016_pull/TMDB_pull_FINAL_2016_dropped_movies_16969.csv")
movies_2017 = pd.read_csv("Resources/2017_pull/TMDB_pull_FINAL_2017_dropped_movies_17367.csv")
movies_2018 = pd.read_csv("Resources/2018_pull/TMDB_pull_FINAL_2018_dropped_movies_17079.csv")
movies_2019 = pd.read_csv("Resources/2019_pull/TMDB_pull_FINAL_2019_dropped_movies_16258.csv")

### Clean Years

#### 2015

In [ ]:
def clean_year(file):
    del file["Unnamed: 0"]
    del file["ID"]
    del file["original_title"]
    return file

def profit(file):
    file["Profit%"] = round((file['revenue']-file['budget'])/file['budget']*100,2)
    
clean_year(movies_2015).head(2)

In [ ]:
movies_2015.reset_index().head()
movies_2015 = movies_2015.drop(index=123)
movies_2015 = movies_2015.rename(columns={'imdb_id':'tconst'})
movies_2015['budget'] = pd.to_numeric(movies_2015['budget'])
movies_2015['revenue'] = pd.to_numeric(movies_2015['revenue'])
profit(movies_2015).head(2)

#### 2016

In [ ]:
clean_year(movies_2016)
movies_2016.reset_index().head()
movies_2016 = movies_2016.drop(index=158)
movies_2016 = movies_2016.rename(columns={'imdb_id':'tconst'})
movies_2016['budget'] = pd.to_numeric(movies_2016['budget'])
movies_2016['revenue'] = pd.to_numeric(movies_2016['revenue'])
profit(movies_2016).head(2)

#### 2017

In [ ]:
clean_year(movies_2017)
movies_2017.reset_index().head()
movies_2017 = movies_2017.rename(columns={'imdb_id':'tconst'})
movies_2017['budget'] = pd.to_numeric(movies_2017['budget'])
movies_2017['revenue'] = pd.to_numeric(movies_2017['revenue'])
profit(movies_2017).head(2)

#### 2018

In [ ]:
clean_year(movies_2018)
movies_2018.reset_index().head()
movies_2018 = movies_2018.drop(index=4)
movies_2018 = movies_2018.rename(columns={'imdb_id':'tconst'})
movies_2018['budget'] = pd.to_numeric(movies_2018['budget'])
movies_2018['revenue'] = pd.to_numeric(movies_2018['revenue'])
profit(movies_2018).head(2)

#### 2019

In [ ]:
clean_year(movies_2019)
movies_2019.reset_index().head()
movies_2019 = movies_2019.drop(index=11)
movies_2019 = movies_2019.drop(index=23)
movies_2019 = movies_2019.drop(index=37)
movies_2019 = movies_2019.rename(columns={'imdb_id':'tconst'})
movies_2019['budget'] = pd.to_numeric(movies_2019['budget'])
movies_2019['revenue'] = pd.to_numeric(movies_2019['revenue'])
profit(movies_2019).head(2)

### Combining Cleaned Years

In [ ]:
movies = movies_2015
movies = movies.append(movies_2016)
movies = movies.append(movies_2017)
movies = movies.append(movies_2018)
movies = movies.append(movies_2019)
movies.count()

In [ ]:
movies = movies.dropna(how='any')
movies.count()

#### Save results as a CSV

In [ ]:
movies.to_csv("Resources/movie_list.csv")
movies.head(2)